# STAGE-2

Stage 2 - Process and clean the raw data and save the processed data in log files

In [1]:
#importing all necessary libraries

import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter

import json
import sys
from nltk.corpus import stopwords, sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yloke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_json('data.json')    #reading json data which is saved in stage-1

In [3]:
data.head()           #top 5 tweets

,created_at,id_str,text,user,place,geo,coordinates
0,2021-12-02 06:19:18+00:00,1466290860794908672,RT @JordanBrace8: Teen suspect wanted in shoot...,"{'id': 388105873, 'id_str': '388105873', 'name...",None,NaN,NaN
1,2021-12-02 06:19:18+00:00,1466290860732010496,RT @AnuSatheesh5: Sharing the Happy news that ...,"{'id': 17996120, 'id_str': '17996120', 'name':...",None,NaN,NaN
2,2021-12-02 06:19:17+00:00,1466290859775770624,RT @premierboxing: 📰: @Gervontaa: The Greatest...,"{'id': 1859507833, 'id_str': '1859507833', 'na...",None,NaN,NaN
3,2021-12-02 06:19:17+00:00,1466290859700199424,RT @Lemniscate_LMN: 🚀🚀🚀Major news!!!! 🚀🚀🚀\n\n✈...,"{'id': 940047078, 'id_str': '940047078', 'name...",None,NaN,NaN
4,2021-12-02 06:19:17+00:00,1466290859230457856,@swati_gs @RakeshTikaitBKU I am looking at a t...,"{'id': 1238396065985986565, 'id_str': '1238396...",None,NaN,NaN


In [4]:
df=data[['text']]

In [5]:
df.head()

,text
0,RT @JordanBrace8: Teen suspect wanted in shoot...
1,RT @AnuSatheesh5: Sharing the Happy news that ...
2,RT @premierboxing: 📰: @Gervontaa: The Greatest...
3,RT @Lemniscate_LMN: 🚀🚀🚀Major news!!!! 🚀🚀🚀\n\n✈...
4,@swati_gs @RakeshTikaitBKU I am looking at a t...


In [49]:
#cleaning the text

#Create a function to clean the tweets
def cleantxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) #Removing @ mentions
    text = re.sub(r'#', '', text) #removing the "#" symbols
    text = re.sub(r'RT[\s]+', '', text) #removing RT
    text = re.sub(r'https?:\/\/\s+', '',text) #removing the hyper link
    text = re.sub(r'(www\.[^\s]+)|(https?://[^\s]+)','',text) # removing emails
        
    return text

#cleaning tweets
df['text']= df['text'].apply(cleantxt)

#show the cleaned text
df

,text
0,: Teen suspect wanted in shooting death of Tem...
1,: Sharing the Happy news that the girl is foun...
2,: 📰: : The Greatest Hits\n\nRead more and watc...
3,_LMN: 🚀🚀🚀Major news!!!! 🚀🚀🚀\n\n✈️Lemniscate &a...
4,_gs I am looking at a time when news reporter...
...,...
995,": After and scoop this morning, we're told..."
996,: I have felt so raw for weeks. The news has b...
997,: Bad news travels fast. TravelFail FallonToni...
998,": Devastating news,a very distinguished creati..."


In [50]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

df['text']= df['text'].apply(remove_emoji)

df

,text
0,: Teen suspect wanted in shooting death of Tem...
1,: Sharing the Happy news that the girl is foun...
2,: : : The Greatest Hits\n\nRead more and watch...
3,_LMN: Major news!!!! \n\nLemniscate &amp; SAVI...
4,_gs I am looking at a time when news reporter...
...,...
995,": After and scoop this morning, we're told..."
996,: I have felt so raw for weeks. The news has b...
997,: Bad news travels fast. TravelFail FallonToni...
998,": Devastating news,a very distinguished creati..."


In [51]:
def tranform_text(text):
    
    text = text.lower()
    text = nltk.word_tokenize(text) 
    #text = re.sub(r'RT[\s]+', '', text) #removing RT
    
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
            
            
    text = y[:]
    y.clear()
    
    custom_list = ['rt']
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    
    return " ".join(y)

df['transformed_text'] = df['text'].apply(tranform_text)

df

,text,transformed_text
0,: Teen suspect wanted in shooting death of Tem...,teen suspect wanted shooting death temple univ...
1,: Sharing the Happy news that the girl is foun...,sharing happy news girl found guruvayur thanks...
2,: : : The Greatest Hits\n\nRead more and watch...,greatest hits read watch highlights website
3,_LMN: Major news!!!! \n\nLemniscate &amp; SAVI...,major news lemniscate amp savior 20 game gift ...
4,_gs I am looking at a time when news reporter...,looking time news reporters channels totally a...
...,...,...
995,": After and scoop this morning, we're told...",scoop morning told attendee drinks 10 dec 18th
996,: I have felt so raw for weeks. The news has b...,felt raw weeks news utterly heart wrenching ev...
997,: Bad news travels fast. TravelFail FallonToni...,bad news travels fast travelfail fallontonight
998,": Devastating news,a very distinguished creati...",devastating news distinguished creative soul f...


In [40]:
# tokenise string

df['tokenise'] = df.apply(lambda row: nltk.word_tokenize(row[1]), axis=1)

df

,text,transformed_text,tokenise
0,: Teen suspect wanted in shooting death of Tem...,teen suspect wanted shooting death temple univ...,"[teen, suspect, wanted, shooting, death, templ..."
1,: Sharing the Happy news that the girl is foun...,sharing happy news girl found guruvayur thanks...,"[sharing, happy, news, girl, found, guruvayur,..."
2,: : : The Greatest Hits\n\nRead more and watch...,greatest hits read watch highlights website,"[greatest, hits, read, watch, highlights, webs..."
3,_LMN: Major news!!!! \n\nLemniscate &amp; SAVI...,major news lemniscate amp savior 20 game gift ...,"[major, news, lemniscate, amp, savior, 20, gam..."
4,_gs I am looking at a time when news reporter...,looking time news reporters channels totally a...,"[looking, time, news, reporters, channels, tot..."
...,...,...,...
995,": After and scoop this morning, we're told...",scoop morning told attendee drinks 10 dec 18th,"[scoop, morning, told, attendee, drinks, 10, d..."
996,: I have felt so raw for weeks. The news has b...,felt raw weeks news utterly heart wrenching ev...,"[felt, raw, weeks, news, utterly, heart, wrenc..."
997,: Bad news travels fast. TravelFail FallonToni...,bad news travels fast travelfail fallontonight,"[bad, news, travels, fast, travelfail, fallont..."
998,": Devastating news,a very distinguished creati...",devastating news distinguished creative soul f...,"[devastating, news, distinguished, creative, s..."


In [41]:
df = df['tokenise']

In [42]:
df

0      [teen, suspect, wanted, shooting, death, templ...
1      [sharing, happy, news, girl, found, guruvayur,...
2      [greatest, hits, read, watch, highlights, webs...
3      [major, news, lemniscate, amp, savior, 20, gam...
4      [looking, time, news, reporters, channels, tot...
                             ...                        
995    [scoop, morning, told, attendee, drinks, 10, d...
996    [felt, raw, weeks, news, utterly, heart, wrenc...
997    [bad, news, travels, fast, travelfail, fallont...
998    [devastating, news, distinguished, creative, s...
999    [child, used, father, gun, kill, four, student...
Name: tokenise, Length: 1000, dtype: object

In [52]:
df1=df['transformed_text']

In [53]:
df1.to_csv("stage_3.csv")          #saving in a csv file to use these tweets in stage-3